# Using quaternions

Quaternions are a widely-used representation in 3D computer graphics and robotics for representing the orientation of rigid bodies in 3D space. They provide a way to avoid issues related to gimbal lock, which can occur when using Euler angles.

Kaolin provides a set of functions to convert between (batches of) quaternions and other 3D rotation representations like 3D rotation matrices, axis-angle representations, and 4D transformation matrices that include a translation and rotation component.

Quaternions are represented in the form:
```
w + x*i + y*j + z*k
```
where `w` is the real component and `x`, `y`, and `z` are the imaginary components of the quaternion.
Kaolin uses the scalar-last (`xyzw`) order, which is common in many graphics libraries.

To learn more about quaternions and how they can be used to represent rotations, check out this [tutorial](https://eater.net/quaternions) from [Grant Sanderson (3blue1brown)](https://www.3blue1brown.com/) and [Ben Eater](https://eater.net/).

Kaolin's functions provide a consistent interface across different 3D rotation representations, making it easy to switch between them when needed while retaining data batching. All operations are torch JIT compiled for acceleration.

In [1]:
# set up the environment
import torch
from kaolin.math import quat

device = 'cuda'

## Quaternion basics

Kaolin supports creating batches of quaternions from different representations. 
Here we create identity quaternions and create a quaternion from a torch tensor using the scalar-last (`xyzw`) representation.

In [2]:
# generate a batch of 2 identity quaternions
identity = quat.quat_identity([2], device=device)
print(identity)

tensor([[0., 0., 0., 1.],
        [0., 0., 0., 1.]], device='cuda:0')


In [3]:
# manually create a batch of 2 quaternions
quats = torch.tensor([[2, 3, 4, 1], [0, -4, -2, -10]], device=device)
print(quats)

# get the real (`w`) component of the two quaternions
print(f"real components:\n{quat.quat_real(quats)}")

# get the imaginary (`xyz`) component of the two quaternions
print(f"imaginary components:\n{quat.quat_imaginary(quats)}")

tensor([[  2,   3,   4,   1],
        [  0,  -4,  -2, -10]], device='cuda:0')
real components:
tensor([[  1],
        [-10]], device='cuda:0')
imaginary components:
tensor([[ 2,  3,  4],
        [ 0, -4, -2]], device='cuda:0')


Kaolin supports common quaternion operations like normalizing to a quaternion to be a valid 3D rotation, multiplying two quaternions, or rotating a 3D point by a quaternion representing a rotation.

In [4]:
# multiply the two quaternions
quats_mul = quat.quat_mul(identity, quats)
print(quats_mul)

tensor([[  2.,   3.,   4.,   1.],
        [  0.,  -4.,  -2., -10.]], device='cuda:0')


In [5]:
# ensure that the quaternion is a valid unit quaternion (norm of 1) and positive real part
valid_3d = quat.quat_unit_positive(quats) 
print(valid_3d)

tensor([[0.3651, 0.5477, 0.7303, 0.1826],
        [-0.0000, 0.3651, 0.1826, 0.9129]], device='cuda:0')


In [6]:
# rotate a point by the quaternion
point = torch.tensor([[1, 2, 3], [1, 2, 3]], device=device)

point_rotated = quat.quat_rotate(rotation=valid_3d, point=point)
print(point_rotated)

tensor([[1.8000, 2.0000, 2.6000],
        [2.0000, 2.6000, 1.8000]], device='cuda:0')


## Quaternion conversions

Kaolin supports conversions among a variety of 3D representations to and from quaternions:

- 3x3 rotation matrix
- 4x4 rotation matrix
- angle-axis (angle and rotation axis)
- Euclidean transformation (4x4)

In [7]:
# convert to 3x3 rotation matrix representation
rot33 = quat.rot33_from_quat(quats)
print(rot33)

tensor([[[-0.6667,  0.1333,  0.7333],
         [ 0.6667, -0.3333,  0.6667],
         [ 0.3333,  0.9333,  0.1333]],

        [[ 0.6667, -0.3333,  0.6667],
         [ 0.3333,  0.9333,  0.1333],
         [-0.6667,  0.1333,  0.7333]]], device='cuda:0')


In [8]:
# convert to angle-axis representation
angle, axis = quat.angle_axis_from_quat(quats)
print(angle, axis)

tensor([[2.7744],
        [0.8411]], device='cuda:0') tensor([[3.7139e-01, 5.5709e-01, 7.4278e-01],
        [2.9200e-07, 8.9443e-01, 4.4721e-01]], device='cuda:0')


In [9]:
# convert the 3x3 rotation matrix to the matching angle-axis representation
angle2, axis2 = quat.angle_axis_from_rot33(rot33)
print(angle2, axis2)

# verify conversions retained the correct values
print(torch.allclose(angle, angle2))
print(torch.allclose(axis, axis2))

tensor([[2.7744],
        [0.8411]], device='cuda:0') tensor([[3.7139e-01, 5.5709e-01, 7.4278e-01],
        [2.9200e-07, 8.9443e-01, 4.4721e-01]], device='cuda:0')
True
True


In [10]:
# convert to 4x4 rotation matrix
rot44 = quat.rot44_from_quat(quats)
print(rot44)

tensor([[[-0.6667,  0.1333,  0.7333,  0.0000],
         [ 0.6667, -0.3333,  0.6667,  0.0000],
         [ 0.3333,  0.9333,  0.1333,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  1.0000]],

        [[ 0.6667, -0.3333,  0.6667,  0.0000],
         [ 0.3333,  0.9333,  0.1333,  0.0000],
         [-0.6667,  0.1333,  0.7333,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  1.0000]]], device='cuda:0')


In [11]:
# compose a Euclidean transform matrix of the quaternion rotation and a translation
euclidean = quat.euclidean_from_rotation_translation(r=quats, t=torch.tensor([[1, 2, 3]]))
print(euclidean)

print(f"rotation component:\n{quat.euclidean_rotation_matrix(euclidean)}")
print(f"translation component:\n{quat.euclidean_translation_vector(euclidean)}")

tensor([[[-0.6667,  0.1333,  0.7333,  1.0000],
         [ 0.6667, -0.3333,  0.6667,  2.0000],
         [ 0.3333,  0.9333,  0.1333,  3.0000],
         [ 0.0000,  0.0000,  0.0000,  1.0000]],

        [[ 0.6667, -0.3333,  0.6667,  1.0000],
         [ 0.3333,  0.9333,  0.1333,  2.0000],
         [-0.6667,  0.1333,  0.7333,  3.0000],
         [ 0.0000,  0.0000,  0.0000,  1.0000]]], device='cuda:0')
rotation component:
tensor([[[-0.6667,  0.1333,  0.7333],
         [ 0.6667, -0.3333,  0.6667],
         [ 0.3333,  0.9333,  0.1333]],

        [[ 0.6667, -0.3333,  0.6667],
         [ 0.3333,  0.9333,  0.1333],
         [-0.6667,  0.1333,  0.7333]]], device='cuda:0')
translation component:
tensor([[1., 2., 3.],
        [1., 2., 3.]], device='cuda:0')
